In [2]:
import heapq
import time
import pandas as pd
from collections import defaultdict

# ============================================================
# 1) DIMACS reader (aggregates parallel arcs) -> paper-friendly
# ============================================================

def read_graph_dimacs_agg(file_path):
    """
    Reads DIMACS-like lines:
      a <source> <target> <weight> <transit_time...>

    - Aggregates parallel arcs: multiple (u,v) become one (u,v) with summed weight
    - Deterministic node mapping (sorted node ids)
    - Deterministic edge order (sorted by (u_idx, v_idx))
    Returns:
      edges_indexed: list[(u_idx, v_idx, w_sum)]
      node_to_index: dict[node_id_str -> int]
      index_to_node: dict[int -> node_id_str]
    """
    agg = defaultdict(float)
    node_ids = set()

    with open(file_path, "r") as f:
        for raw in f:
            line = raw.strip()
            if not line:
                continue
            if line.startswith(("c", "p")):
                continue
            if not line.startswith("a"):
                continue

            parts = line.split()
            if len(parts) < 4:
                continue

            u = parts[1]
            v = parts[2]
            try:
                w = float(parts[3])
            except ValueError:
                continue

            node_ids.add(u)
            node_ids.add(v)
            agg[(u, v)] += w

    node_list = sorted(node_ids)
    node_to_index = {node: i for i, node in enumerate(node_list)}
    index_to_node = {i: node for node, i in node_to_index.items()}

    edges_indexed = [(node_to_index[u], node_to_index[v], float(w_sum))
                     for (u, v), w_sum in agg.items()]
    edges_indexed.sort(key=lambda e: (e[0], e[1]))
    return edges_indexed, node_to_index, index_to_node


# ============================================================
# 2) Fast graph representation: edge IDs + active flags
#    (NO adjacency rebuild per iteration)
# ============================================================

def build_eid_graph(edges_indexed, n_nodes, tol=1e-12):
    """
    Converts edges_indexed into edge-id arrays for speed.

    Returns:
      U, V: list[int] endpoints per edge id
      W0:  list[float] original weights
      W:   list[float] mutable reduced weights
      active: bytearray (1 if W>tol else 0)
      adj: list[list[int]] adjacency lists of edge IDs (outgoing)
    """
    m = len(edges_indexed)
    U = [0] * m
    V = [0] * m
    W0 = [0.0] * m
    W = [0.0] * m
    active = bytearray(m)
    adj = [[] for _ in range(n_nodes)]

    for eid, (u, v, w) in enumerate(edges_indexed):
        U[eid] = u
        V[eid] = v
        W0[eid] = float(w)
        W[eid] = float(w)
        if w > tol:
            active[eid] = 1
        adj[u].append(eid)

    # edges_indexed was sorted by (u,v), so for each u, adj[u] is already deterministic.
    return U, V, W0, W, active, adj


# ============================================================
# 3) Cycle finding in the ACTIVE graph (edge-id adjacency)
#    (NO O(n) array reinitialization; reset only touched nodes)
# ============================================================

def find_any_cycle_eids(n_nodes, adj, U, V, active):
    """
    Finds any directed cycle in the active graph.
    Returns: list of edge IDs forming a directed cycle, or None if acyclic.

    Implementation details:
      - iterative DFS
      - skips inactive edges
      - resets only visited nodes (not full O(n) reset)
    """
    state = bytearray(n_nodes)  # 0=unvisited, 1=visiting, 2=done (for THIS call)
    parent = [-1] * n_nodes
    parent_eid = [-1] * n_nodes
    next_ptr = [0] * n_nodes
    visited_nodes = []

    def reset():
        for x in visited_nodes:
            state[x] = 0
            parent[x] = -1
            parent_eid[x] = -1
            next_ptr[x] = 0
        visited_nodes.clear()

    for s in range(n_nodes):
        if state[s] != 0:
            continue

        stack = [s]
        state[s] = 1
        visited_nodes.append(s)

        while stack:
            u = stack[-1]
            i = next_ptr[u]

            # advance to next ACTIVE outgoing edge
            out = adj[u]
            while i < len(out) and active[out[i]] == 0:
                i += 1
            next_ptr[u] = i

            if i >= len(out):
                state[u] = 2
                stack.pop()
                continue

            eid = out[i]
            v = V[eid]
            next_ptr[u] = i + 1  # move forward

            if state[v] == 0:
                parent[v] = u
                parent_eid[v] = eid
                state[v] = 1
                visited_nodes.append(v)
                stack.append(v)
            elif state[v] == 1:
                # back-edge u->v forms a cycle
                cycle_eids = [eid]
                cur = u
                # follow parents from u back to v
                while cur != v:
                    pe = parent_eid[cur]
                    if pe == -1:
                        break
                    cycle_eids.append(pe)
                    cur = parent[cur]
                    if cur == -1:
                        break

                if cur == v:
                    cycle_eids.reverse()
                    reset()
                    return cycle_eids
                # else: something inconsistent (shouldn't happen), continue

        # finished component; continue to next s

    reset()
    return None


# ============================================================
# 4) Topological order on ACTIVE edges (deterministic)
# ============================================================

def topo_order_active(n_nodes, adj, V, active):
    """
    Kahn topo sort on active graph.
    Returns:
      order: list of node indices
      rank: list[int] rank[node] in 0..n-1
    Raises if graph is not acyclic.
    """
    indeg = [0] * n_nodes
    for u in range(n_nodes):
        for eid in adj[u]:
            if active[eid]:
                indeg[V[eid]] += 1

    heap = []
    for i in range(n_nodes):
        if indeg[i] == 0:
            heapq.heappush(heap, i)

    order = []
    while heap:
        u = heapq.heappop(heap)
        order.append(u)
        for eid in adj[u]:
            if not active[eid]:
                continue
            v = V[eid]
            indeg[v] -= 1
            if indeg[v] == 0:
                heapq.heappush(heap, v)

    if len(order) != n_nodes:
        raise RuntimeError("Topological sort failed: active graph is not acyclic (unexpected).")

    rank = [0] * n_nodes
    for r, node in enumerate(order):
        rank[node] = r
    return order, rank


# ============================================================
# 5) Reachability test for add-back: cycle iff v reaches u
#    (prune by topo rank interval)
# ============================================================

def make_reachability_checker(n_nodes, adj, V, active):
    """
    Returns a closure reachable(src, target, rank, rank_limit).
    Uses stamp-based visited to avoid clearing per call.
    """
    visited = [0] * n_nodes
    stamp = 0

    def reachable(src, target, rank, rank_limit):
        nonlocal stamp
        stamp += 1
        st = stamp

        # quick cases
        if src == target:
            return True
        if rank[src] > rank_limit:
            return False  # pruned by interval

        stack = [src]
        visited[src] = st

        while stack:
            x = stack.pop()
            for eid in adj[x]:
                if not active[eid]:
                    continue
                y = V[eid]
                if rank[y] > rank_limit:
                    continue  # prune by topo interval
                if y == target:
                    return True
                if visited[y] != st:
                    visited[y] = st
                    stack.append(y)
        return False

    return reachable


# ============================================================
# 6) Paper algorithm: Local-ratio cycle reductions + add-back
#    Improvements:
#      - no adjacency rebuild
#      - cycle finder resets only touched nodes
#      - add-back: heavy edges first
#      - add-back: O(1) accept if rank[u] < rank[v]
#      - else: reachability v->u with topo interval pruning
#      - recompute topo ranks only when needed
# ============================================================

def local_ratio_fas_fast(edges_indexed, n_nodes, tol=1e-12):
    """
    Returns:
      removed_eids: set[int] edge IDs in the final minimal FAS (under heuristic order)
      U,V,W0: edge arrays (for reporting)
      active, adj: final active DAG
    """
    U, V, W0, W, active, adj = build_eid_graph(edges_indexed, n_nodes, tol=tol)

    removed_eids = set()

    # ---- Phase 1: cycle reductions ----
    while True:
        cyc = find_any_cycle_eids(n_nodes, adj, U, V, active)
        if cyc is None:
            break

        # eps = min reduced weight on this cycle
        eps = None
        for eid in cyc:
            w = W[eid]
            if eps is None or w < eps:
                eps = w

        if eps is None or eps <= tol:
            # numerical safety: deactivate the first edge on the cycle
            eid0 = cyc[0]
            if active[eid0]:
                active[eid0] = 0
                W[eid0] = 0.0
                removed_eids.add(eid0)
            continue

        # subtract eps from all cycle edges
        for eid in cyc:
            new_w = W[eid] - eps
            W[eid] = new_w
            if new_w <= tol and active[eid]:
                active[eid] = 0
                W[eid] = 0.0
                removed_eids.add(eid)

    # At this point, active graph should be a DAG.
    # ---- Phase 2: add-back (heavy first) ----
    removed_list = sorted(
        list(removed_eids),
        key=lambda eid: (-W0[eid], U[eid], V[eid])
    )

    # initial topo ranks
    _, rank = topo_order_active(n_nodes, adj, V, active)
    reachable = make_reachability_checker(n_nodes, adj, V, active)

    for eid in removed_list:
        u = U[eid]
        v = V[eid]

        # Fast accept: respects current topo order => cannot form cycle
        if rank[u] < rank[v]:
            active[eid] = 1
            removed_eids.discard(eid)
            continue

        # Otherwise, adding u->v creates a cycle iff v reaches u
        # Prune search to nodes with rank <= rank[u]
        if not reachable(v, u, rank=rank, rank_limit=rank[u]):
            active[eid] = 1
            removed_eids.discard(eid)

            # rank may no longer be a valid topo order after adding a "backward" edge.
            # For correctness of future O(1) checks and pruning, recompute ranks now.
            _, rank = topo_order_active(n_nodes, adj, V, active)
        # else: keep it removed

    return removed_eids, U, V, W0, active, adj


# ============================================================
# 7) End-to-end: DIMACS -> paper FAS -> topo ranking CSV
# ============================================================

def paper_fas_ranking_from_dimacs_fast(dimacs_path, output_ranking_csv_path, tol=1e-12):
    """
    Produces a ranking CSV (Node ID, Order) using the paper algorithm with speedups.
    Compatible with your forward/backward sum checker (rank_u < rank_v).
    """
    edges_indexed, node_to_index, index_to_node = read_graph_dimacs_agg(dimacs_path)
    n = len(node_to_index)

    removed_eids, U, V, W0, active, adj = local_ratio_fas_fast(edges_indexed, n, tol=tol)

    # Final topo order gives ranking
    order, rank = topo_order_active(n, adj, V, active)

    # Write ranking: Node ID, Order
    rows = [{"Node ID": str(index_to_node[i]).strip(), "Order": int(rank[i])} for i in range(n)]
    rows.sort(key=lambda r: r["Order"])
    pd.DataFrame(rows).to_csv(output_ranking_csv_path, index=False)

    # Convert removed edges to (u,v) pairs for reporting
    F_removed_pairs = {(U[eid], V[eid]) for eid in removed_eids}

    scores = {i: int(rank[i]) for i in range(n)}
    return edges_indexed, node_to_index, index_to_node, scores, F_removed_pairs


# ============================================================
# 8) Forward/backward evaluation helper (your semantics)
# ============================================================

def compute_forward_backward(edges_indexed, scores):
    total_w = 0.0
    fw = 0.0
    for u, v, w in edges_indexed:
        total_w += w
        if scores[u] < scores[v]:
            fw += w
    bw = total_w - fw
    return total_w, fw, bw


# ============================================================
# 9) Main
# ============================================================

if __name__ == "__main__":
    edge_file = "/mmfs1/home/sv96/Feedback-arc-set-paper/datasets/s38417.d"
    out_csv = edge_file.replace(".d", "") + "_paper_fas_ranking.csv"

    t0 = time.perf_counter()

    edges_indexed, node_to_index, index_to_node, scores, F_removed = paper_fas_ranking_from_dimacs_fast(
        dimacs_path=edge_file,
        output_ranking_csv_path=out_csv,
        tol=1e-12
    )

    total_w, fw, bw = compute_forward_backward(edges_indexed, scores)

    elapsed_sec = time.perf_counter() - t0

    print(f"✅ Wrote ranking: {out_csv}")
    print(f"Graph: n={len(node_to_index)} nodes, m={len(edges_indexed)} edges (after aggregation)")
    print(f"Total Weight: {total_w:.6f}")
    print(f"Forward Weight: {fw:.6f}")
    print(f"Backward Weight: {bw:.6f}")
    print(f"Forward Ratio: {fw/total_w:.6f}")
    print(f"Removed edges in minimal FAS (count): {len(F_removed)}")
    print(f"⏱️ Running time: {elapsed_sec:.3f} seconds ({elapsed_sec/60.0:.3f} minutes)")


✅ Wrote ranking: /mmfs1/home/sv96/Feedback-arc-set-paper/datasets/s38417_paper_fas_ranking.csv
Graph: n=24255 nodes, m=34876 edges (after aggregation)
Total Weight: 52252938.000000
Forward Weight: 51903935.000000
Backward Weight: 349003.000000
Forward Ratio: 0.993321
Removed edges in minimal FAS (count): 1123
⏱️ Running time: 7.204 seconds (0.120 minutes)


In [40]:
# ============================================================
# WMSF implementation (paper049) for FAIR comparison (UPDATED)
# - Same DIMACS reader (read_graph_dimacs_agg)
# - Same output format: ranking CSV with columns (Node ID, Order)
# - Deterministic: fixed ordering ties (eid, u, v)
# - Uses fast edge-ID + active flags (no adjacency rebuild)
#
# IMPORTANT UPDATES (paper-faithful):
#   1) For SINGLE-SCC graphs: run BOTH L1 and L2 and keep the better (min BW).
#   2) Pipeline per SCC: removeArcs -> MinimizeFas -> StabilizeFas -> MinimizeFas
#      (matches paper text: stabilization can destroy minimality -> minimize again)
# ============================================================

import heapq
import math
import time
import pandas as pd
from collections import defaultdict, deque

# ------------------------------------------------------------------
# Reuse YOUR functions if already defined above:
#   read_graph_dimacs_agg
#   topo_order_active
#   make_reachability_checker
# ------------------------------------------------------------------

def compute_forward_backward(edges_indexed, scores):
    total_w = 0.0
    fw = 0.0
    for u, v, w in edges_indexed:
        total_w += w
        if scores[u] < scores[v]:
            fw += w
    bw = total_w - fw
    return total_w, fw, bw


def build_eid_graph_inout(edges_indexed, n_nodes, tol=1e-12):
    """
    Edge-ID arrays with BOTH out- and in-adjacency (for stabilization step).
    """
    m = len(edges_indexed)
    U = [0] * m
    V = [0] * m
    W0 = [0.0] * m
    active = bytearray(m)
    out_adj = [[] for _ in range(n_nodes)]
    in_adj  = [[] for _ in range(n_nodes)]

    for eid, (u, v, w) in enumerate(edges_indexed):
        U[eid] = u
        V[eid] = v
        W0[eid] = float(w)
        if w > tol:
            active[eid] = 1
        out_adj[u].append(eid)
        in_adj[v].append(eid)

    return U, V, W0, active, out_adj, in_adj


# ============================================================
# SCC decomposition (Kosaraju), deterministic adjacency order
# ============================================================

def kosaraju_scc(n_nodes, edges_indexed):
    """
    Returns list of SCCs, each as list of vertices.
    Deterministic if edges_indexed is deterministic.
    """
    outN = [[] for _ in range(n_nodes)]
    inN  = [[] for _ in range(n_nodes)]
    for (u, v, _w) in edges_indexed:
        outN[u].append(v)
        inN[v].append(u)

    seen = bytearray(n_nodes)
    order = []

    # iterative DFS finish order
    for s in range(n_nodes):
        if seen[s]:
            continue
        stack = [(s, 0)]
        seen[s] = 1
        while stack:
            u, it = stack[-1]
            if it >= len(outN[u]):
                order.append(u)
                stack.pop()
                continue
            v = outN[u][it]
            stack[-1] = (u, it + 1)
            if not seen[v]:
                seen[v] = 1
                stack.append((v, 0))

    comp_id = [-1] * n_nodes
    comps = []
    cid = 0

    # reverse graph DFS
    for s in reversed(order):
        if comp_id[s] != -1:
            continue
        comp = []
        stack = [s]
        comp_id[s] = cid
        while stack:
            u = stack.pop()
            comp.append(u)
            for v in inN[u]:
                if comp_id[v] == -1:
                    comp_id[v] = cid
                    stack.append(v)
        comps.append(comp)
        cid += 1

    return comps, comp_id


def topo_order_active(n_nodes, adj, V, active):
    """
    Kahn topo sort on active graph.
    Returns:
      order: list of node indices
      rank: list[int] rank[node] in 0..n-1
    Raises if graph is not acyclic.
    """
    indeg = [0] * n_nodes
    for u in range(n_nodes):
        for eid in adj[u]:
            if active[eid]:
                indeg[V[eid]] += 1

    heap = []
    for i in range(n_nodes):
        if indeg[i] == 0:
            heapq.heappush(heap, i)

    order = []
    while heap:
        u = heapq.heappop(heap)
        order.append(u)
        for eid in adj[u]:
            if not active[eid]:
                continue
            v = V[eid]
            indeg[v] -= 1
            if indeg[v] == 0:
                heapq.heappush(heap, v)

    if len(order) != n_nodes:
        raise RuntimeError("Topological sort failed: active graph is not acyclic.")

    rank = [0] * n_nodes
    for r, node in enumerate(order):
        rank[node] = r
    return order, rank


def edges_by_scc(edges_indexed, comp_id):
    """
    Returns: dict[scc_index] -> list of edge tuples (u,v,w,eid_global)
    Only edges INSIDE the SCC.
    """
    by = defaultdict(list)
    for eid, (u, v, w) in enumerate(edges_indexed):
        cu = comp_id[u]
        cv = comp_id[v]
        if cu == cv:
            by[cu].append((u, v, w, eid))
    return by


# ============================================================
# WMSF core steps on one SCC (paper049)
#   removeArcs -> MinimizeFas -> StabilizeFas -> MinimizeFas
# ============================================================

def make_reachability_checker(n_nodes, adj, V, active):
    """
    Returns a closure reachable(src, target, rank, rank_limit).
    Uses stamp-based visited to avoid clearing per call.
    """
    visited = [0] * n_nodes
    stamp = 0

    def reachable(src, target, rank, rank_limit):
        nonlocal stamp
        stamp += 1
        st = stamp

        if src == target:
            return True
        if rank[src] > rank_limit:
            return False

        stack = [src]
        visited[src] = st

        while stack:
            x = stack.pop()
            for eid in adj[x]:
                if not active[eid]:
                    continue
                y = V[eid]
                if rank[y] > rank_limit:
                    continue
                if y == target:
                    return True
                if visited[y] != st:
                    visited[y] = st
                    stack.append(y)
        return False

    return reachable


def _is_acyclic_active(n_nodes, out_adj, V, active):
    try:
        _order, rank = topo_order_active(n_nodes, out_adj, V, active)
        return True, rank
    except RuntimeError:
        return False, None


def wmsf_removeArcs_scc(n_nodes, U, V, W0, active, out_adj, in_adj, ordering="L2", tol=1e-12):
    """
    paper049 Section 2.1:
      - 2-cycles preprocessing: remove earlier arc in ordering
      - temporarily remove safe arcs (not in F)
      - delete arcs by ordering; check acyclicity every alpha deletions (alpha=|A|/|V|)
    """
    # Win/Wout for L2 (computed on current SCC graph)
    Win = [0.0] * n_nodes
    Wout = [0.0] * n_nodes
    for eid in range(len(U)):
        if not active[eid]:
            continue
        u = U[eid]; v = V[eid]
        w = W0[eid]
        Wout[u] += w
        Win[v]  += w

    eids = [eid for eid in range(len(U)) if active[eid]]

    if ordering.upper() == "L1":
        eids.sort(key=lambda eid: (W0[eid], U[eid], V[eid], eid))
    else:
        def keyL2(eid):
            denom = Win[U[eid]] + Wout[V[eid]]
            if denom <= 0.0:
                denom = 1.0
            return (W0[eid] / denom, W0[eid], U[eid], V[eid], eid)
        eids.sort(key=keyL2)

    F = set()

    # 2-cycles preprocessing: remove earlier edge in ordering
    pos = {eid: i for i, eid in enumerate(eids)}
    pair_to_eid = {}
    for eid in eids:
        pair_to_eid[(U[eid], V[eid])] = eid

    for eid in eids:
        if not active[eid]:
            continue
        u, v = U[eid], V[eid]
        rev = pair_to_eid.get((v, u), None)
        if rev is None or not active[rev]:
            continue
        if pos[eid] < pos[rev]:
            active[eid] = 0
            F.add(eid)

    # safe arcs trimming
    indeg = [0] * n_nodes
    outdeg = [0] * n_nodes
    for u in range(n_nodes):
        for eid in out_adj[u]:
            if active[eid]:
                outdeg[u] += 1
                indeg[V[eid]] += 1

    q = deque()
    for eid in range(len(U)):
        if not active[eid]:
            continue
        u = U[eid]; v = V[eid]
        if indeg[u] == 0 or outdeg[v] == 0:
            q.append(eid)

    safe_tmp = []
    while q:
        eid = q.popleft()
        if not active[eid]:
            continue
        u = U[eid]; v = V[eid]
        if not (indeg[u] == 0 or outdeg[v] == 0):
            continue

        active[eid] = 0
        safe_tmp.append(eid)

        outdeg[u] -= 1
        indeg[v] -= 1

        for ee in out_adj[u]:
            if active[ee]:
                q.append(ee)
        for ee in in_adj[v]:
            if active[ee]:
                q.append(ee)

    # delete by ordering, check every alpha deletions
    m_act = 0
    for eid in range(len(U)):
        if active[eid]:
            m_act += 1
    alpha = max(1, m_act // max(1, n_nodes))
    since_check = 0

    for eid in eids:
        if not active[eid]:
            continue
        active[eid] = 0
        F.add(eid)
        since_check += 1
        if since_check >= alpha:
            since_check = 0
            ok, _rank = _is_acyclic_active(n_nodes, out_adj, V, active)
            if ok:
                break

    ok, _rank = _is_acyclic_active(n_nodes, out_adj, V, active)
    if not ok:
        for eid in eids:
            if not active[eid]:
                continue
            active[eid] = 0
            F.add(eid)
            ok, _rank = _is_acyclic_active(n_nodes, out_adj, V, active)
            if ok:
                break

    # restore safe arcs
    for eid in safe_tmp:
        active[eid] = 1

    return F, safe_tmp


def wmsf_minimizeFas_scc(n_nodes, U, V, W0, active, out_adj, F, tol=1e-12):
    """
    paper049 Section 2.2: heavy-first add-back.
    Efficient implementation:
      - O(1) accept if rank[u] < rank[v]
      - else accept iff not reachable(v,u) with topo-rank pruning
      - recompute topo ranks only when we accept a backward edge
    """
    for eid in F:
        active[eid] = 0

    _, rank = topo_order_active(n_nodes, out_adj, V, active)
    reachable = make_reachability_checker(n_nodes, out_adj, V, active)

    cand = sorted(list(F), key=lambda eid: (-W0[eid], U[eid], V[eid], eid))

    for eid in cand:
        u = U[eid]; v = V[eid]
        if rank[u] < rank[v]:
            active[eid] = 1
            F.discard(eid)
            continue
        if not reachable(v, u, rank=rank, rank_limit=rank[u]):
            active[eid] = 1
            F.discard(eid)
            _, rank = topo_order_active(n_nodes, out_adj, V, active)

    return F


def wmsf_stabilizeFas_scc(n_nodes, U, V, W0, active, out_adj, in_adj, F, tol=1e-12):
    """
    paper049 Definition 2.1 + Section 2.3.
    """
    WinG = [0.0] * n_nodes
    WoutG = [0.0] * n_nodes
    for eid in range(len(U)):
        u = U[eid]; v = V[eid]
        w = W0[eid]
        WoutG[u] += w
        WinG[v]  += w

    max_passes = max(1, int(math.log2(max(2, n_nodes))))
    for _ in range(max_passes):
        order, _rank = topo_order_active(n_nodes, out_adj, V, active)
        changed = False

        for v in order:
            WinStar = 0.0
            for eid in in_adj[v]:
                if active[eid]:
                    WinStar += W0[eid]
            WoutStar = 0.0
            for eid in out_adj[v]:
                if active[eid]:
                    WoutStar += W0[eid]

            removed_in  = WinG[v]  - WinStar
            removed_out = WoutG[v] - WoutStar

            if removed_in > WoutStar + tol:
                # remove all remaining outgoing
                for eid in out_adj[v]:
                    if active[eid]:
                        active[eid] = 0
                        F.add(eid)
                        changed = True
                # add back all removed incoming
                for eid in in_adj[v]:
                    if (not active[eid]) and (eid in F):
                        active[eid] = 1
                        F.discard(eid)
                        changed = True

            elif removed_out > WinStar + tol:
                # remove all remaining incoming
                for eid in in_adj[v]:
                    if active[eid]:
                        active[eid] = 0
                        F.add(eid)
                        changed = True
                # add back all removed outgoing
                for eid in out_adj[v]:
                    if (not active[eid]) and (eid in F):
                        active[eid] = 1
                        F.discard(eid)
                        changed = True

        if not changed:
            break

    return F


def _sync_active_from_F(active, m, F):
    for eid in range(m):
        active[eid] = 0 if (eid in F) else 1


def _wmsf_pipeline_scc(k, U2, V2, W02, active2, out2, in2, ordering, tol=1e-12):
    """
    paper-faithful SCC pipeline:
      removeArcs -> Minimize -> Stabilize -> Minimize
    """
    F2, _safe = wmsf_removeArcs_scc(k, U2, V2, W02, active2, out2, in2, ordering=ordering, tol=tol)
    for e in F2:
        active2[e] = 0

    F2 = wmsf_minimizeFas_scc(k, U2, V2, W02, active2, out2, F2, tol=tol)
    _sync_active_from_F(active2, len(U2), F2)

    F2 = wmsf_stabilizeFas_scc(k, U2, V2, W02, active2, out2, in2, F2, tol=tol)
    _sync_active_from_F(active2, len(U2), F2)

    F2 = wmsf_minimizeFas_scc(k, U2, V2, W02, active2, out2, F2, tol=tol)
    _sync_active_from_F(active2, len(U2), F2)

    return F2, active2


def _build_local_scc_graph(verts, e_list, tol=1e-12):
    verts_sorted = sorted(verts)
    loc = {v: i for i, v in enumerate(verts_sorted)}
    k = len(verts_sorted)

    edges_local = [(loc[u], loc[v], w, eid_global) for (u, v, w, eid_global) in e_list]
    m_scc = len(edges_local)

    U2 = [0] * m_scc
    V2 = [0] * m_scc
    W02 = [0.0] * m_scc
    eidG = [0] * m_scc
    active2 = bytearray(m_scc)
    out2 = [[] for _ in range(k)]
    in2  = [[] for _ in range(k)]

    for eid2, (uu, vv, ww, eg) in enumerate(edges_local):
        U2[eid2] = uu
        V2[eid2] = vv
        W02[eid2] = float(ww)
        eidG[eid2] = eg
        if ww > tol:
            active2[eid2] = 1
        out2[uu].append(eid2)
        in2[vv].append(eid2)

    return k, U2, V2, W02, eidG, active2, out2, in2


def wmsf_ranking_from_dimacs_fast(dimacs_path, output_ranking_csv_path, ordering="L2", tol=1e-12):
    """
    UPDATED entry point (drop-in replacement):
      - If entire graph is a SINGLE SCC: run BOTH L1 and L2 and keep min BW.
      - Otherwise: run one ordering per SCC (ordering arg), SCC-union.
    """
    edges_indexed, node_to_index, index_to_node = read_graph_dimacs_agg(dimacs_path)
    n = len(node_to_index)

    comps, comp_id = kosaraju_scc(n, edges_indexed)
    by_scc = edges_by_scc(edges_indexed, comp_id)

    U, V, W0, active_glob0, out_adj_glob, in_adj_glob = build_eid_graph_inout(edges_indexed, n, tol=tol)

    # detect "whole graph is one SCC"
    nontrivial = [c for c in comps if len(c) > 1]
    whole_single_scc = (len(nontrivial) == 1 and len(nontrivial[0]) == n)

    def run_one(ordering_choice):
        active_glob = bytearray(active_glob0)
        F_global = set()

        for scc_idx, verts in enumerate(comps):
            e_list = by_scc.get(scc_idx, [])

            if len(verts) <= 1:
                # remove self-loops
                for (u, v, w, eid) in e_list:
                    if u == v and w > tol:
                        F_global.add(eid)
                        active_glob[eid] = 0
                continue

            if not e_list:
                continue

            k, U2, V2, W02, eidG, active2, out2, in2 = _build_local_scc_graph(verts, e_list, tol=tol)

            F2, _active2 = _wmsf_pipeline_scc(
                k, U2, V2, W02, active2, out2, in2,
                ordering=ordering_choice, tol=tol
            )

            for eid2 in F2:
                eg = eidG[eid2]
                F_global.add(eg)
                active_glob[eg] = 0

        order_nodes, rank = topo_order_active(n, out_adj_glob, V, active_glob)
        scores = {i: int(rank[i]) for i in range(n)}
        _tot, _fw, bw = compute_forward_backward(edges_indexed, scores)
        return bw, scores, F_global, active_glob

    # whole-graph single SCC: best of L1 & L2
    if whole_single_scc:
        bw1, scores1, F1, active1 = run_one("L1")
        bw2, scores2, F2, active2 = run_one("L2")
        if bw1 <= bw2:
            best_scores, best_F, best_active = scores1, F1, active1
        else:
            best_scores, best_F, best_active = scores2, F2, active2
    else:
        bw, best_scores, best_F, best_active = run_one(ordering)

    # write ranking CSV
    rows = [{"Node ID": str(index_to_node[i]).strip(), "Order": int(best_scores[i])} for i in range(n)]
    rows.sort(key=lambda r: r["Order"])
    pd.DataFrame(rows).to_csv(output_ranking_csv_path, index=False)

    F_removed_pairs = {(U[eid], V[eid]) for eid in best_F}
    return edges_indexed, node_to_index, index_to_node, best_scores, F_removed_pairs


# ============================================================
# Example usage (mirrors your main)
# ============================================================
if __name__ == "__main__":
    edge_file = "/mmfs1/home/sv96/Feedback-arc-set-paper/datasets/s38417.d"
    out_csv = edge_file.replace(".d", "") + "_wmsf_ranking.csv"

    t0 = time.perf_counter()

    edges_indexed, node_to_index, index_to_node, scores, F_removed = wmsf_ranking_from_dimacs_fast(
        dimacs_path=edge_file,
        output_ranking_csv_path=out_csv,
        ordering="L2",   # used only if graph is NOT a single SCC
        tol=1e-12
    )

    total_w, fw, bw = compute_forward_backward(edges_indexed, scores)
    elapsed_sec = time.perf_counter() - t0

    print(f"✅ Wrote ranking: {out_csv}")
    print(f"Graph: n={len(node_to_index)} nodes, m={len(edges_indexed)} edges (after aggregation)")
    print(f"Total Weight: {total_w:.6f}")
    print(f"Forward Weight: {fw:.6f}")
    print(f"Backward Weight: {bw:.6f}")
    print(f"Forward Ratio: {fw/total_w:.6f}")
    print(f"Removed edges (count): {len(F_removed)}")
    print(f"⏱️ Running time: {elapsed_sec:.3f} seconds ({elapsed_sec/60.0:.3f} minutes)")


✅ Wrote ranking: /mmfs1/home/sv96/Feedback-arc-set-paper/datasets/s38417_wmsf_ranking.csv
Graph: n=24255 nodes, m=34876 edges (after aggregation)
Total Weight: 52252938.000000
Forward Weight: 51886198.000000
Backward Weight: 366740.000000
Forward Ratio: 0.992981
Removed edges (count): 1146
⏱️ Running time: 25.030 seconds (0.417 minutes)
